In [180]:
### metadata redactions
data = read.csv('./synapse/expression_matrices//qc_sample_metadata.csv')

data$molecule_h5 <- NULL

data$age_death_redacted = ifelse(data$age_death>90, '90+', round(data$age_death))

data$age_death <- NULL
data$X <- NULL

data$pmi_redacted = round(data$pmi)
data$pmi <- NULL

data$annotations <- NULL
data$keep_cells_gaussian <- NULL
data$predictions <- NULL
data$gaussian_liklihood <- NULL
data$leiden_clusters <- NULL

data$index <- NULL

data$annotations <- data$annotations2
data$annotations2 <- NULL

write.csv(data, './synapse/expression_matrices/qc_rowData.csv', row.names=FALSE)

In [181]:
### format the gene names
data = read.csv('./synapse/expression_matrices/qc_gene_names.csv')
data$X = NULL
colnames(data) = c('gene_name', 'Ensembl_ID')
write.csv(data, './synapse/expression_matrices/qc_colData.csv', row.names=FALSE)

In [69]:
### check md5sums for batch 4819F
d1 = read.csv('../logs/md5sum_ABCA7-lof_BMC_10x-4819F.txt', sep = '', header = FALSE)
d1$name = unlist(lapply(strsplit(as.character(d1$V2), '/'), function(x) x[length(x)]))
d1$name = paste0(unlist(strsplit(d1$name, '.fastq.gz')), '_batch_4819F.fastq.gz')

d2 = read.csv('./synapse/fastq_files/md5sums.txt', sep = '', header = FALSE)
colnames(d2) = c('V1', 'name')
combined = merge(d1, d2, by = 'name')
unique(as.character(combined$V1.x)==as.character(combined$V1.y))

In [79]:
### check md5sums for batch 4826F
d1 = read.csv('../logs/md5sum_ABCA7-lof_BMC_10x-4826F.txt', sep = '', header = FALSE)
d1$name = unlist(lapply(strsplit(as.character(d1$V2), '/'), function(x) x[length(x)]))
d1$name = paste0(unlist(strsplit(d1$name, '.fastq.gz')), '_batch_4826F.fastq.gz')

d2 = read.csv('./synapse/fastq_files/md5sums.txt', sep = '', header = FALSE)
colnames(d2) = c('V1', 'name')
combined = merge(d1, d2, by = 'name')
unique(as.character(combined$V1.x)==as.character(combined$V1.y))

[1] TRUE

In [89]:
# save individual metadata
data = read.csv('../raw_data/metadata/single_cell_individual_metadata.csv')

data$molecule_h5 <- NULL

data$age_death_redacted = ifelse(data$age_death>90, '90+', round(data$age_death))

data$age_death <- NULL
data$X <- NULL

data$pmi_redacted = round(data$pmi)
data$pmi <- NULL

write.csv(data, './synapse/fastq_files/individual_metadata_redacted.csv', row.names=FALSE)

In [15]:
# save LoF variant info 

metadata = read.csv('../raw_data/metadata/single_cell_individual_metadata.csv')

variant_subset_info = read.csv('../processed_data/variants/variant_subset_info.csv', check.names=FALSE)
variant_subset_geno = read.csv('../processed_data/variants/variant_subset_geno.csv', check.names=FALSE)

variant_subset_geno$X <- NULL
variant_subset_info$X <- NULL

index = variant_subset_info$LOF!=''
index2 = variant_subset_info$HGVS_C=='c.5570+5G>C'
index_all = index2 | index
variant_subset_info_LOF = variant_subset_info[index_all,]
variant_subset_geno_LOF = variant_subset_geno[index_all, ]
variant_subset_geno_LOF_our_samples = variant_subset_geno_LOF[,as.character(metadata$WGSid)]

index = variant_subset_info_LOF$GENE=='ABCA7'
variant_subset_info_LOF = variant_subset_info_LOF[index,]
variant_subset_geno_LOF_our_samples = variant_subset_geno_LOF_our_samples[index,]

write.csv(cbind(variant_subset_info_LOF, variant_subset_geno_LOF_our_samples), '../open_data/synapse/fastq_files/ABCA7lof_variant_info.csv', row.names=FALSE)

In [179]:
sum(rowSums(variant_subset_geno_LOF_our_samples=='0/1'))

[1] 12

In [189]:
# format lipidomic sample code
data = read.csv('./synapse/lipidomics/lipidomic_sample_code.csv')

data$Position <- NULL
data$Tube <- NULL
data$Tube_plus_brain <- NULL
data$Tube_minus_brain <- NULL
data$vial_plus_brain <- NULL
data$vial_minus_brain <- NULL

colnames(data) = c('projid', 'ABCA7LoF', 'SampleID')

write.csv(data, '../open_data/synapse/lipidomics/lipidomic_sample_code.csv', row.names=FALSE)

In [36]:
# collecting data for manifest
data = read.csv('./all_files.txt', header = FALSE)
colnames(data) = c('path')

folders = unlist(lapply(strsplit(as.character(data$path), './synapse/'), function(x) strsplit(x[[2]], '/')[[1]][1]))
data$parent = ifelse(folders=='fastq_files', 'syn53468871', ifelse(folders=='lipidomics', 'syn53468872','syn53468873'))
                        
data$fileFormat = unlist(lapply(strsplit(as.character(data$path), '[.]'), function(x) x[length(x)]))

# get sampleIDs
IDs = unlist(lapply(strsplit(as.character(data$path), '/'), function(x) ifelse(startsWith(x[4], 'D'), strsplit(x[4], '_')[[1]][1], strsplit(x[4], '_')[[1]][3])))
IDs = unlist(lapply(strsplit(IDs, '.raw'), function(x) x[[1]][1]))
                    
metadata = read.csv('./synapse/fastq_files/individual_metadata_redacted.csv')
rownames(metadata) = metadata$projid
metadata2 = read.csv('./synapse/lipidomics/lipidomic_sample_code.csv')
df2 = as.data.frame(cbind(as.character(metadata2$SampleID), as.character(metadata[as.character(metadata2$projid),'individualID'])))
colnames(df2) = c('ID', 'Name')
df1 = metadata[,c('sample_id', 'individualID')]
colnames(df1) = c('ID', 'Name')
rownames(df1) <- NULL

code = rbind(df2, df1)
rownames(code) = code$ID

temp = ifelse(!startsWith(as.character(code$ID), 'D'), strsplit(as.character(code$ID), '[.]'), as.character(code$ID))
rownames(code) = unlist(lapply(temp, function(x) paste(x, collapse='')))
data$individualID = code[IDs, 'Name']
                               
 
# other                            
data$specimenID = NA                               
data$isMultiSpecimen = ifelse(data$fileFormat=='csv' | data$fileFormat=='txt' | data$fileFormat=='xlsx' | data$fileFormat=='mtx', TRUE, FALSE)
data$isModelSystem = FALSE
data$consortium = 'CDCP'                                
data$study = 'ROSMAP-ABCA7lof'
data$grant = NA
                                
data$assay = ifelse((data$fileFormat=='gz' | data$fileFormat=='mtx'), 'rnaSeq', ifelse((data$fileFormat=='raw' |data$fileFormat=='xlsx'), 'lipidomics', NA))
data$resourceType = ifelse(data$fileFormat=='csv' | data$fileFormat=='txt', 'metadata', 'experimentalData')
data$dataSubtype = ifelse((data$fileFormat=='gz' | data$fileFormat=='raw'), 'raw', ifelse((data$fileFormat=='xlsx' | data$fileFormat=='mtx' | data$fileFormat=='txt'), 'processed', 'metadata'))
                                
data$metadataType = NA
data$analysisType = NA
data$chromosome = NA
#data$path = sprintf("'%s'",data$path)
                               
#write.csv(data, './manifest.csv', row.names=FALSE)
                               
write.table(data, "./manifest.txt", sep="\t", row.names=FALSE, col.names=TRUE, quote=FALSE)


In [17]:
# format the var info data
data = read.csv('./synapse/fastq_files/ABCA7lof_variant_info.csv', check.names=FALSE)
data = data[,!colnames(data)%in%c('Var.1', 'ALT_1', 'ALT_2', 'REF_y', 'ALT')]
colnames(data)[colnames(data)=='REF_x'] = 'REF'
colnames(data)[colnames(data)=='ALT_0'] = 'ALT'
write.csv(data, './synapse/fastq_files/ABCA7lof_variant_info.csv', row.names=FALSE)